In [65]:
import pandas as pd
import numpy as np
from collections import defaultdict
lis_ = defaultdict(list)


In [5]:
infile = "nlp_text.txt"
outfile = "cleaned_file.txt"

delete_list = ["="]
fin = open(infile)
fout = open(outfile, "w+")
for line in fin:
    for word in delete_list:
        line = line.replace(word, "")
    fout.write(line)
fin.close()
fout.close()

In [6]:
with open("cleaned_file.txt", "rt") as fin:
    with open("nlp2.txt", "wt") as fout:
        for line in fin:
            fout.write(line.replace('||', '='))

In [100]:
import xlrd
cancer_text = pd.read_excel('cancerText.xlsx', sheetname='Sheet2')

cancer_text.columns = ['ID','Text']
cancer_data = pd.read_csv('train_aa_extra.csv')
cancer_data.drop('Unnamed: 0', axis=1, inplace=True)
frames = [cancer_data, cancer_text]

In [44]:
cancerTotal = pd.merge(cancer_data, cancer_text, on='ID', how='inner')

In [178]:
###Splitting data into train and test sets
cancerTotal.Text
cancerTotal.Class
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cancerTotal.Text, cancerTotal.Class, test_size=0.3, random_state=42)


In [101]:
class1 = cancerTotal[cancerTotal['Class'] == 1]
class2 = cancerTotal[cancerTotal['Class'] == 2]
class2_subset = class2[class2.Text.duplicated() == False]
class1_subset = class1[class1.Text.duplicated() == False]


In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0, stop_words = 'english')
tf_class2 = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0, stop_words = 'english')

In [103]:
tfidf_matrix =  tf.fit_transform(class1_subset.Text)
tfidf_matrix2 = tf_class2.fit_transform(class2_subset.Text)
feature_names = tf.get_feature_names() 
feature_names2 = tf_class2.get_feature_names()



1294529

In [104]:
tfidf_matrix
tfidf_matrix2
dense = tfidf_matrix.todense()
dense2 = tfidf_matrix2.todense()
len(dense[0].tolist()[0])

1294529

In [106]:
class_words = dense[0].tolist()[0]
class_words2 = dense2[0].tolist()[0]
phrase_scores = [pair for pair in zip(range(0, len(class_words)), class_words) if pair[1] > 0]
phrase_scores2 = [pair for pair in zip(range(0, len(class_words2)), class_words2) if pair[1] > 0]


In [107]:
sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)
sorted_phrase_scores2 = sorted(phrase_scores2, key = lambda t: t[1] * -1)
#for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:30]:
#   print('{0: <20} {1}'.format(phrase, score))
for phrase, score in [(feature_names2[word_id], score) for (word_id, score) in sorted_phrase_scores2][:30]:
   print('{0: <20} {1}'.format(phrase, score))

cbl                  0.681035781489112
loh                  0.11330215180338506
cbl mutations        0.09367960594390486
h94y                 0.09367960594390486
s80n                 0.09367960594390486
s80n h94y            0.09367960594390486
taiwanese            0.09093943725377807
q249e                0.08698820551934024
w802                 0.08698820551934024
egfr                 0.08064303109027307
lung                 0.07502802746146157
met egfr             0.07494590933659642
ring finger          0.07388829276869469
african              0.06579849261914447
met                  0.0652875285784702
mutations            0.06198582764456129
h94y q249e           0.0602226038210817
q249e w802           0.0602226038210817
s80n h94y q249e      0.0602226038210817
tkb                  0.05620943200244731
finger               0.05557519515547279
cbl mutants          0.05353120339651707
cbl mutation         0.05353120339651707
h94y q249e w802      0.05353120339651707
cell                 0

In [98]:
sorted(phrase_scores, key=lambda t: t[1] * -1)[:5]

[(271523, 0.6652867899292048),
 (473721, 0.2872293230762133),
 (367239, 0.25478807860878017),
 (271543, 0.24239131105947767),
 (1141169, 0.10208322411945689)]

In [112]:
sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)
for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:20]:
   print('{0: <20} {1}'.format(phrase, score))
print('----------------------------------------')
sorted_phrase_scores = sorted(phrase_scores2, key=lambda t: t[1] * -1)
for phrase, score in [(feature_names2[word_id], score) for (word_id, score) in sorted_phrase_scores2][:20]:
   print('{0: <20} {1}'.format(phrase, score))

cdk10                0.6652867899292048
ets2                 0.2872293230762133
cyclin               0.25478807860878017
cdk10 cyclin         0.24239131105947767
star                 0.10208322411945689
6his                 0.08251619099897113
star syndrome        0.07735892906153544
fig                  0.06746923265014898
ets2 protein         0.06704440518666405
gst cdk10            0.06704440518666405
myc cdk10            0.061887143249228345
cdk10 wt             0.05672988131179266
strepii              0.05672988131179266
v5 6his              0.05672988131179266
mcf7                 0.053708193263610544
mcf7 cells           0.052801667647994946
silencing            0.05245632914881128
cdk10 wt kd          0.051572619374356955
cyclin silencing     0.051572619374356955
fam58a               0.051572619374356955
----------------------------------------
cbl                  0.681035781489112
loh                  0.11330215180338506
cbl mutations        0.09367960594390486
h94y          

In [113]:
###Naive Bayes Training



In [185]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape


(2324, 97319)

In [186]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2324, 97319)

In [190]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [191]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
                     ])
text_clf = text_clf.fit(X_train, y_train)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.44032096288866601

In [193]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, n_iter=5, random_state=42)),
])
_ = text_clf_svm.fit(X_train, y_train)
predicted_svm = text_clf_svm.predict(X_test)
np.mean(predicted_svm == y_test)

0.60682046138415247

In [198]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 3)],
               'tfidf__use_idf': (True, False),
               'clf__alpha': (1e-2, 1e-3),
              }
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
gs_clf.best_score_
gs_clf.best_params_

In [ ]:
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': (True, False),
               'clf-svm__alpha': (1e-2, 1e-3),}
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(twenty_train.data, twenty_train.target)
gs_clf_svm.best_score_
gs_clf_svm.best_params_